In [1]:
!pip install openai
!pip install langchain
!pip install chromadb
!pip install duckduckgo-search
!pip install chardet
!pip install tiktoken
!wget https://raw.githubusercontent.com/stockmarkteam/ner-wikipedia-dataset/main/ner.json
!pip install spacy[cuda100]
!pip install -U ginza https://github.com/megagonlabs/ginza/releases/download/latest/ja_ginza_electra-latest-with-model.tar.gz
!pip install -U ja_ginza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.

# 宣言

In [ ]:
import json
import os
import time
import spacy
import pandas as pd
from tqdm.auto import tqdm
import random

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import messages_to_dict
from langchain.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.chains import RetrievalQA
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.vectorstores import Chroma

from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.tools import Tool
from langchain.tools import DuckDuckGoSearchRun

gpu = spacy.prefer_gpu()
#print('GPU:', gpu)
nlp = spacy.load('ja_ginza_electra')

os.environ["OPENAI_API_KEY"] = "xxx"
lim = ChatOpenAI(model_name="gpt-3.5-turbo-1106",temperature=0,max_tokens=512)
memory = ConversationBufferWindowMemory(k=2,return_messages=True)

search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name="duckduckgo-search",
        func=search.run,
        description="useful for when you need to search for latest information in web",
        )
    ]

agent = initialize_agent(
    tools=tools,
    llm=lim,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # AgentType.ZERO_SHOT_REACT_DESCRIPTIONとはなんだ
    verbose=True
)

template = """
#以下に示す条件を守ってください
- あなたは24歳女性のマイです。
- 文体は話し言葉で
- おてんば娘風な口調で話してください
- 短い2文で出力してください
- 相手が返答しやすいように文章を出力してください
"""

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(template),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])
conversation = ConversationChain(llm=lim, memory=memory, prompt=prompt)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


# 関数(raise_topicがmain部分)

In [15]:
def select_topic(texts):
  docs = list(nlp.pipe(texts))
  pred_ents = []

  for idx, doc in enumerate(docs):
    pred_ent = [ent.text for ent in doc.ents]

    # 要素を重複させずにリストを結合
    pred_ents = list(set(pred_ents) | set(pred_ent))

  choice_pred = random.choice(pred_ents)
  return choice_pred

def topic_agent(topic_source_word):
  result = agent.run(f"""
            {topic_source_word}についてその詳細を教えてください。
            """)
  time.sleep(0.1)
  # Systempromptとして指示を与えたい
  command = f"以下の{topic_source_word}についての情報\n\n{result}\n\nを使用して、トラック運転手との会話が弾むような話題を投げかけてください。\n話題を投げかける際は、会話の内容そのトピックの詳細に踏み込んでいるとなおよいです。"
  response = conversation.predict(input=command)
  return response

def raise_topic(texts):
  topic = select_topic(texts)
  response = topic_agent(topic)
  return response

# ここからはテスト部分だから気にせず

In [8]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(template),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])
conversation = ConversationChain(llm=lim, memory=memory, prompt=prompt)
command = input("You: ")
while True:
  response = conversation.predict(input=command)
  print(f"AI: {response}")
  time.sleep(0.1)
  command = input("You: ")
  if command == "exit":
    break

buffer = memory.load_memory_variables({})
texts = []
for historys in buffer["history"]:
  texts.append(historys.content)

You: 私は人間ですか
AI: もちろん！人間だよ！何か質問ある？
You: いいえ，私は宇宙人です
AI: へー、宇宙人なんだ！じゃあ、地球に来た理由は何？
You: exit


In [14]:
raise_topic(texts)
print(f"AI: {response}")

['人間']


> Entering new AgentExecutor chain...
I should use duckduckgo-search to find information about 人間 (human beings).
Action: duckduckgo-search
Action Input: "人間についての情報"
Observation: 記憶のメカニズム. 記憶とは？. 情報を保管してのちに思い出す機能. 記憶とは、過去の経験や取り入れた情報を一度脳内の貯蔵庫に保管して、のちにそれを思い出す機能のことです。. 記憶はヒトに限らず生物がもつ基本的な働きで、広い意味でいえば私 ... 中村 幸司 解説委員. ヒトゲノム、つまりヒトの遺伝情報について、すべて読み取る国際的なプロジェクト「ヒトゲノム計画」が遺伝情報の解読を ... 「ヒトゲノム計画」がヒトの遺伝情報の解読を完了したと宣言してから20年以上を経たいま、その「解読」と「理解」との間に大きな隔たりが ... 人間の内にある二つの情報系統 コロナ禍は、人間が、分子担体情報（生体内）と電子担体情報（脳→半導体→インターネット内）の作用の下に在り、ウィルスやICTも、そうした情報担体のプール内で進化していること（情報展開過程）を示し、その進化現象の加速度をいっそう増した。 ところ ... 専門家や米テスラのイーロン・マスク氏といったテック企業のトップらは今年3月にも、次世代AIの開発を止めるよう求める公開書簡を発表して ...
Thought:This search result is not providing the specific information I need about 人間 (human beings). I should try a different search query.
Action: duckduckgo-search
Action Input: "人間の特徴とは"
Observation: 4. 「人間の定義」まとめ. この記事では先人が考えた6つの人間の定義について解説しました。 みなさんはどの定義に共感できましたか？ 最後に今日の内容を、覚え方も含めて表にまとめましたので、おさらいしてください。 「人間性」とは、「人間らしさ、人間的な性質」のこ